In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_471480/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_36_5,0.999050,0.659087,0.616041,0.998916,0.000401,0.202381,0.268971,0.000591,0.009491,0.020019,0.997763,0.020196,145.644394,323.497293,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_36_4,0.999049,0.659321,0.616739,0.998938,0.000401,0.202242,0.268482,0.000579,0.009506,0.020024,0.997762,0.020202,145.643346,323.496245,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_36_2,0.999048,0.661710,0.622769,0.999109,0.000401,0.200823,0.264258,0.000486,0.009508,0.020032,0.997760,0.020210,145.641772,323.494671,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_36_3,0.999047,0.659332,0.616553,0.998958,0.000402,0.202235,0.268612,0.000568,0.009535,0.020048,0.997756,0.020227,145.638424,323.491324,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_36_1,0.999038,0.661678,0.622468,0.999121,0.000406,0.200842,0.264469,0.000479,0.009557,0.020144,0.997735,0.020323,145.619332,323.472231,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_4,0.609453,0.412452,0.402976,0.624376,0.164693,0.348794,0.418228,0.158631,0.233717,0.405823,0.080587,0.409431,133.607349,311.460249,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_3,0.577938,0.386490,0.381301,0.591398,0.177982,0.364206,0.433412,0.172558,0.244366,0.421879,0.006396,0.425630,133.452143,311.305042,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.500444,0.343359,0.321575,0.498945,0.210661,0.389810,0.475251,0.211602,0.262067,0.458978,-0.176037,0.463058,133.115009,310.967908,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,0.395913,0.270565,0.240294,0.390729,0.254742,0.433024,0.532190,0.257303,0.287048,0.504719,-0.422122,0.509206,132.735010,310.587909,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
